In [ ]:
import pandas as pd
import geopandas as gpd

# Load crossings shapefile
crossings_path = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston\crossings.shp"
crossings = gpd.read_file(crossings_path)

# Create new column for xLTS
crossings['xLTS'] = None

# Convert relevant columns to numeric (handle possible NaNs)
numeric_cols = ['CRS_SPEED', 'CRS_ISLAND', 'CRS_ADT', 'CRS_LANES']
crossings[numeric_cols] = crossings[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [29]:
# Assign xLTS=1 for cases where approach has priority
crossings.loc[crossings['CONTROL'].isin(['signal', 'stop', 'implied_stop']), 'xLTS'] = 1

In [30]:
# Path crossings at unsignalized intersections

for idx, row in crossings.iterrows():
    if pd.notna(row['xLTS']):
        continue
    
    speed = row['CRS_SPEED']
    rrfb = row['RRFB']
    island = row['CRS_ISLAND']
    adt = row['CRS_ADT']
    path_xing = row['PATH_XING']

    if path_xing == "True":
        if speed <= 38.5:
            if rrfb:
                xLTS = 1
            elif adt <= 8_000:
                xLTS = 1
        elif speed <= 28.5:
            if island:
                xLTS = 1

        if 'xLTS' in locals():
            crossings.at[idx, 'xLTS'] = xLTS

In [31]:
# Assign xLTS for other unsignalized crossings
for idx, row in crossings.iterrows():
    if pd.notna(row['xLTS']):
        continue

    speed = row['CRS_SPEED']
    island = row['CRS_ISLAND']
    adt = row['CRS_ADT']
    lanes = row['CRS_LANES'] * 2  # Two times the number of lanes per direction

    # Initialize xLTS only when conditions are met
    if speed <= 38.5 and island == 1:
        if adt < 12_000:
            xLTS = 1
        elif adt < 24_000:
            xLTS = 2
        else:
            xLTS = 3
    elif speed <= 38.5 and island == 0:
        if lanes <= 3:
            if adt < 9_000:
                xLTS = 1
            elif adt < 17_000:
                xLTS = 2
            else:
                xLTS = 3
        else:
            if adt < 6_000:
                xLTS = 1
            elif adt < 12_000:
                xLTS = 2
            elif adt < 19_000:
                xLTS = 3
            else:
                xLTS = 4
    elif speed > 38.5 and island == 1:
        if adt < 8_000:
            xLTS = 1
        elif adt < 17_000:
            xLTS = 2
        else:
            xLTS = 3
    elif speed > 38.5 and island == 0:
        if lanes <= 3:
            if adt < 6_000:
                xLTS = 1
            elif adt < 12_000:
                xLTS = 2
            elif adt < 18_000:
                xLTS = 3
            else:
                xLTS = 4
        else:
            if adt < 10_000:
                xLTS = 2
            elif adt < 14_000:
                xLTS = 3
            else:
                xLTS = 4
    else:
        continue  # Skip if no conditions are met

    # Assign xLTS to the DataFrame
    crossings.at[idx, 'xLTS'] = xLTS

In [32]:
# Freeway Ramp Triggers (98 for on ramp and 99 for off ramp)
crossings.loc[~crossings['CONTROL'].isin(['signal', 'stop', 'implied_stop']) & 
              (crossings['CRS_qNoAcc'].isin([98, 99])), 'xLTS'] = 2

In [33]:
# Save the updated shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

In [34]:
print(crossings['xLTS'].value_counts(dropna=False))

xLTS
1    39989
2     1964
Name: count, dtype: int64
